# Bibliotheken importieren
Importieren Sie die notwendigen Bibliotheken wie Pandas, NumPy und Matplotlib.

In [ ]:
# Bibliotheken importieren

import pandas as pd  # Importieren von Pandas für die Datenanalyse
import numpy as np  # Importieren von NumPy für numerische Operationen
import matplotlib.pyplot as plt  # Importieren von Matplotlib für die Datenvisualisierung

# Daten laden
Laden Sie die Daten aus einer CSV-Datei oder einer anderen Datenquelle.

In [ ]:
# Daten laden

# Laden der Daten aus einer CSV-Datei
df = pd.read_csv('pfad/zur/datei.csv')

# Anzeigen der ersten fünf Zeilen des DataFrames
df.head()

# Daten vorverarbeiten
Bereinigen und transformieren Sie die Daten, um sie für die Analyse vorzubereiten.

In [ ]:
# Daten vorverarbeiten

# Überprüfen auf fehlende Werte
fehlende_werte = df.isnull().sum()
print("Fehlende Werte pro Spalte:\n", fehlende_werte)

# Entfernen von Zeilen mit fehlenden Werten
df = df.dropna()

# Entfernen von Duplikaten
df = df.drop_duplicates()

# Umwandeln von Datentypen (Beispiel: Umwandeln einer Spalte in den Datentyp 'datetime')
df['datumsspalte'] = pd.to_datetime(df['datumsspalte'])

# Skalieren von numerischen Daten (Beispiel: Normalisierung)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
numerische_spalten = ['spalte1', 'spalte2', 'spalte3']
df[numerische_spalten] = scaler.fit_transform(df[numerische_spalten])

# Anzeigen der ersten fünf Zeilen des bereinigten DataFrames
df.head()

# Daten analysieren
Führen Sie verschiedene Analysen auf den Daten durch, um Erkenntnisse zu gewinnen.

In [ ]:
# Daten analysieren

# Beschreibende Statistik der numerischen Spalten anzeigen
statistik = df.describe()
print("Beschreibende Statistik:\n", statistik)

# Korrelation zwischen den numerischen Spalten berechnen
korrelation = df.corr()
print("Korrelationsmatrix:\n", korrelation)

# Histogramme der numerischen Spalten erstellen
df[numerische_spalten].hist(bins=15, figsize=(15, 6), layout=(2, 3))
plt.tight_layout()
plt.show()

# Boxplots der numerischen Spalten erstellen
df[numerische_spalten].plot(kind='box', subplots=True, layout=(2, 3), figsize=(15, 6))
plt.tight_layout()
plt.show()

# Scatterplot-Matrix der numerischen Spalten erstellen
pd.plotting.scatter_matrix(df[numerische_spalten], figsize=(15, 10))
plt.show()

# Ergebnisse visualisieren
Visualisieren Sie die Ergebnisse der Analyse mit Diagrammen und Grafiken.

In [ ]:
# Ergebnisse visualisieren

# Balkendiagramm der durchschnittlichen Werte der numerischen Spalten
df[numerische_spalten].mean().plot(kind='bar', figsize=(10, 6))
plt.title('Durchschnittliche Werte der numerischen Spalten')
plt.xlabel('Spalten')
plt.ylabel('Durchschnittlicher Wert')
plt.show()

# Liniendiagramm der Zeitreihe (Beispiel: 'datumsspalte' vs. 'spalte1')
plt.figure(figsize=(10, 6))
plt.plot(df['datumsspalte'], df['spalte1'], marker='o')
plt.title('Zeitreihe von spalte1')
plt.xlabel('Datum')
plt.ylabel('Wert von spalte1')
plt.grid(True)
plt.show()

# Heatmap der Korrelationsmatrix
plt.figure(figsize=(10, 8))
sns.heatmap(korrelation, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Heatmap der Korrelationsmatrix')
plt.show()

# Kreisdiagramm der Verteilung einer kategorialen Spalte (Beispiel: 'kategoriespalte')
df['kategoriespalte'].value_counts().plot(kind='pie', autopct='%1.1f%%', figsize=(8, 8))
plt.title('Verteilung der kategoriespalte')
plt.ylabel('')
plt.show()